<a href="https://colab.research.google.com/github/alexontour/snippets/blob/main/snip_mysql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Author: Alexander Kollmann, 10/2022**

---

**Funktion**

Verbindung zu einer MySQL-DB herstellen

CRUD-Operationen ausführen

---



**Referenzen**

https://www.w3schools.com/sql/default.asp



---

In [34]:
# Notwendige Bibliotheken installieren um MySQL-Datenbank ansprechen zu können
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
python3-dev is already the newest version (3.6.7-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
# Importieren der notwendiggen Bibliotheken
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [36]:
# Verbindung zu der Datenbank aufbauen. Bei dem Beisiel handelt es sich um eine Datenbank auf einem freien SQL-Server. D.h. KEINE persönliche Information abspeichern.

#MYSQL_HOSTNAME = 'sql11.freemysqlhosting.net'

MYSQL_HOSTNAME = 'remotemysql.com'
MYSQL_USER = 'ddv67SeI3z'
MYSQL_PASSWORD = 'i934PvTXFJ'
MYSQL_DATABASE = 'ddv67SeI3z'

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}/{MYSQL_DATABASE}'

con = create_engine(connection_string)

In [37]:
# INSERT (gerade bei DB-Operationen wird empfohlen ein Error-Handling (try-except) zu verwenden um mögliche Fehler "abzufangen")
try:
    query="INSERT INTO  `ddv67SeI3z`.`pat` (`pat_id` ,`pat_vorname` ,`pat_nachname` ,`pat_gebdat`)  VALUES('bpk4','hugo','boss','2022-02-22')"
    
    id=con.execute(query)
    
    print("Rows Added  = ",id.rowcount)
    
except SQLAlchemyError as e:
        error = str(e.__dict__['orig'])
        print(error)

Rows Added  =  1


In [38]:
# CREATE/ INSERT (mehrere)

try:
    query="INSERT INTO  `ddv67SeI3z`.`pat` (`pat_id` ,`pat_vorname` ,`pat_nachname` ,`pat_gebdat`)  VALUES(%s,%s,%s,%s)"
    data=[('bpk1','vn1','nn1','1950-12-01'),
            ('bpk2','vn2','nn2','1950-12-02'),
            ('bpk3','vn3','nn3','1950-12-03')]
    
    id=con.execute(query,data)
    
    print("Rows Added  = ",id.rowcount)
    
except SQLAlchemyError as e:
        error = str(e.__dict__['orig'])
        print(error)

(1062, "Duplicate entry 'bpk1' for key 'PRIMARY'")


In [39]:
# SELECT / READ (einfach)

import pandas as pd

query = """SELECT * FROM pat;""".format(MYSQL_DATABASE)

pat_df = pd.read_sql(query, con)
pat_df

,pat_id,pat_vorname,pat_nachname,pat_gebdat
0,bpk1,vn1,nn1,1950-12-01
1,bpk2,vn2,nn2,1950-12-02
2,bpk3,vn3,nn3,1950-12-03
3,bpk4,hugo,boss,2022-02-22


In [40]:
# SELECT / READ (join)
# Welche PAtienten haben eine "offene, stationäre Kontaktbestätigung" in einer Krankenanstalt (d.h. diese Patienten sind zzt. stationär aufgenommen)
# Gute Erklärung der verschiedenen JOINs
# https://www.w3schools.com/sql/sql_join.asp 

import pandas as pd

query = """select `pat`.`pat_nachname` AS `pat_nachname`,`pat`.`pat_vorname` AS `pat_vorname`,`gda`.`gda_bezeichnung` AS `gda_bezeichnung` 
           from ((`kbs` join `pat` on(`kbs`.`pat_id` = `pat`.`pat_id`)) join `gda` on(`kbs`.`gda_id` = `gda`.`gda_id`)) 
           where `kbs`.`kbs_type` = 'stat' and `kbs`.`kbs_end` is null and `gda`.`gda_rolle` = 'Krankenanstalt'""".format(MYSQL_DATABASE)

pat_df = pd.read_sql(query, con)
pat_df

,pat_nachname,pat_vorname,gda_bezeichnung
0,nn1,vn1,Salzburg LKA


In [41]:
# UPDATE

import sqlalchemy as sa

# Reflect the database table into an object  
tbl = sa.Table('pat', sa.MetaData(), autoload_with=con)
# Create an update object
upd = sa.update(tbl).where(tbl.c.pat_id == 'bpk4').values(pat_vorname='a', pat_nachname='k')

# The "begin" context manager will automatically commit on exit
with con.begin() as conn:
    conn.execute(upd)

In [42]:
# Check - UPDATE
import pandas as pd

query = """SELECT * FROM pat;""".format(MYSQL_DATABASE)

pat_df = pd.read_sql(query, con)
pat_df

,pat_id,pat_vorname,pat_nachname,pat_gebdat
0,bpk1,vn1,nn1,1950-12-01
1,bpk2,vn2,nn2,1950-12-02
2,bpk3,vn3,nn3,1950-12-03
3,bpk4,a,k,2022-02-22


In [43]:
# DELETE (Tip: Delete kann man auch über einen Parameter "ausblenden" oder "inaktiv" abbeilden)

import sqlalchemy as sa

# Reflect the database table into an object  
tbl = sa.Table('pat', sa.MetaData(), autoload_with=con)
# Create an update object
dele = sa.delete(tbl).where(tbl.c.pat_id == 'bpk4')

# The "begin" context manager will automatically commit on exit
with con.begin() as conn:
    conn.execute(dele)

In [44]:
# Check - DELETE
import pandas as pd

query = """SELECT * FROM pat;""".format(MYSQL_DATABASE)

pat_df = pd.read_sql(query, con)
pat_df

,pat_id,pat_vorname,pat_nachname,pat_gebdat
0,bpk1,vn1,nn1,1950-12-01
1,bpk2,vn2,nn2,1950-12-02
2,bpk3,vn3,nn3,1950-12-03
